<h1>Keras einfaches Word Embedding</h1>

Word Embeddings sind eine Form der Repräsentation von Wörtern in Zahlen, mit dem ein Model trainiert werden kann, um NLP Aufgaben zu lösen.

Mit Embeddings können Relationen gut abgebildet werden. <br>
Eine Methode Embeddings zu erstellen ist, durch Trainieren eines Models, das uns am Ende Embeddings liefert.
- Erstelle Fake-Problem, um eventuell Embeddings zu bekommen, die die Wörter gut repräsentiert

Als Einstieg in die Thematik sollen mittels einfachen Reviews Embeddings erstellt werden. 

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
import tensorflow as tf

# Mögliche Reviews:
reviews = ['nice food',
        'amazing restaurant',
        'too good',
        'just loved it!',
        'will go again',
        'horrible food',
        'never go there',
        'poor service',
        'poor quality',
        'needs improvement']

In [15]:
# Positiver oder negativer Review. 
y_truth = np.array([1,1,1,1,1,0,0,0,0,0])

Als Erstes muss das Vokabular erstellt werden. Jedes Wort bekommt eine Zahl, Relationen können damit nicht abgebildet werden.

Keras liefert eine Methode, mit dem das einfach umsetzbar ist.

In [2]:
# Input Text | n
# - Hat auch Filterfunktion. 
one_hot("eins zwei drei", 10)  # Vokabular Größe n, hier 10.

[2, 6, 7]

Damit bekommt jedes Wort eine Zahl.

Damit erstellen wir eine On-Hot-Encode Matrix. <br>
Diese Vektoren werden später für die Multiplikation benötigt. 

In [3]:
# Einfach als Liste.
ohe_review = [one_hot(wort, 10)  for wort in reviews ]
ohe_review

[[6, 3],
 [4, 5],
 [6, 2],
 [8, 2, 3],
 [4, 6, 9],
 [2, 3],
 [3, 6, 2],
 [5, 5],
 [5, 2],
 [5, 5]]

Das neurale Netz hat eine feste Größe, um eine verschiedene Anzahl von Wörtern (in einem Satz) zu verarbeiten, wird Padding angewendet, um den Rest mit Nullen zu füllen.

Dabei kann Keras uns auch wieder helfen, es schnell und sauber umzusetzen. 

In [4]:
# Keras übernimmt Padding.
# - Als Input: Review, Max. Länge des Satzes, Padding Typ. 
ohe_review_pad = tf.keras.preprocessing.sequence.pad_sequences(\
               ohe_review, maxlen=5, padding='post')
ohe_review_pad

array([[6, 3, 0, 0, 0],
       [4, 5, 0, 0, 0],
       [6, 2, 0, 0, 0],
       [8, 2, 3, 0, 0],
       [4, 6, 9, 0, 0],
       [2, 3, 0, 0, 0],
       [3, 6, 2, 0, 0],
       [5, 5, 0, 0, 0],
       [5, 2, 0, 0, 0],
       [5, 5, 0, 0, 0]])

Der Rest wurde mit Nullen aufgefüllt. 

Für die Embeddings muss jetzt eine Vektorgröße festgelegt werden. 

In [13]:
# Vektorgröße:
vec_size = 6

# Model:
# - Siehe: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding
model = tf.keras.Sequential([
    # Vokabulargröße | Vektorgröße  | Name
    tf.keras.layers.Embedding(10, vec_size, name="emb_1"),   # Embedding Layer
    tf.keras.layers.Flatten(),  # Flattern der Vektoren (10 x 6).
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

model.compile(
    optimizer = 'adam',
    loss      = 'binary_crossentropy',
    metrics   = ['accuracy']
)

In [16]:
model.fit(ohe_review_pad, y_truth, epochs=30)

Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 715ms/step - accuracy: 0.5000 - loss: 0.6901
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5000 - loss: 0.6890
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5000 - loss: 0.6880
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5000 - loss: 0.6870
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7000 - loss: 0.6860
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7000 - loss: 0.6850
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7000 - loss: 0.6840
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8000 - loss: 0.6829
Epoch 9/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8000 - loss: 0.6819
Epoch 10/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8000 - loss: 0.6809
Epoch 11/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8000 - loss: 0.6798
Epoch 12/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8000 - loss: 0.6788


Die Embeddings, die wir brauchen, befinden sich in dem Model selber. <br>
Die Paramter des Models liefern und die Embeddings.


In [20]:
# Shape:
# - 10 x 6, da Vokabular 10, und Vektoren 6
model.get_layer('emb_1').get_weights()[0].shape

(10, 6)

In [21]:
model.get_layer('emb_1').get_weights()[0]

array([[ 0.01813056, -0.0137063 ,  0.04271284,  0.02789754, -0.03505011,
        -0.02280086],
       [-0.01580471,  0.03044316, -0.02784019, -0.02579024,  0.01480803,
        -0.00919183],
       [-0.08440124, -0.01927229,  0.02501341,  0.01622639, -0.03229559,
        -0.04692887],
       [ 0.0414257 ,  0.04064957, -0.03529285,  0.01843199, -0.007053  ,
        -0.00471885],
       [ 0.01832675, -0.08308379,  0.01550242, -0.07139666, -0.03885722,
         0.08597784],
       [-0.09916144,  0.07317287, -0.09895664,  0.00464139,  0.05587775,
        -0.01652415],
       [ 0.02392933, -0.06716495, -0.01238208, -0.03219894, -0.03007421,
         0.06286578],
       [-0.03673953, -0.00168223,  0.04505447, -0.01007368,  0.00092851,
         0.02447336],
       [ 0.07097796, -0.03203809,  0.02632191, -0.07793061, -0.02839172,
         0.02507659],
       [ 0.05204882, -0.03395499,  0.07831118,  0.02311316, -0.00902729,
        -0.0138696 ]], dtype=float32)

In [23]:
emb = model.get_layer('emb_1').get_weights()[0]

Jetzt können wir uns die Embeddings für bestimmte Wörter anschauen.

In [24]:
# Nice:6, amazing: 4
emb[6]

array([ 0.02392933, -0.06716495, -0.01238208, -0.03219894, -0.03007421,
        0.06286578], dtype=float32)

In [25]:
emb[4]

array([ 0.01832675, -0.08308379,  0.01550242, -0.07139666, -0.03885722,
        0.08597784], dtype=float32)

In [26]:
# poor: 5
emb[5]

array([-0.09916144,  0.07317287, -0.09895664,  0.00464139,  0.05587775,
       -0.01652415], dtype=float32)